In [ ]:
import os
import sys
import pdb
import math
import random
import argparse
import numpy as np
import matplotlib
import matplotlib.path as mpath
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
from sklearn.mixture import GaussianMixture


def error_handler(e):
    print(
        'Error on file {} line {}'.format(sys.exc_info()[-1].tb_frame.f_code.co_filename, sys.exc_info()[-1].tb_lineno),
        type(e).__name__, e)


def parse_data(txt_file):
    action_list = {}
    ego_list = {}
    exos_list = {}
    coll_bool_list = {}
    ego_path_list = {}
    pred_car_list = {}
    pred_exo_list = {}
    trial_list = {}
    depth_list = {}

    exo_count = 0
    start_recording = False

    with open(txt_file, 'r') as f:
        for line in f:
            if 'Round 0 Step' in line:
                line_split = line.split('Round 0 Step ', 1)[1]
                cur_step = int(line_split.split('-', 1)[0])
                start_recording = True
            if not start_recording:
                continue
            try:
                if "car pos / heading / vel" in line:  # ego_car info
                    speed = float(line.split(' ')[12])
                    heading = float(line.split(' ')[10])
                    pos_x = float(line.split(' ')[7].replace('(', '').replace(',', ''))
                    pos_y = float(line.split(' ')[8].replace(')', '').replace(',', ''))
                    bb_x = float(line.split(' ')[15])
                    bb_y = float(line.split(' ')[16])

                    pos = [pos_x, pos_y]

                    agent_dict = {'pos': [pos_x, pos_y],
                                  'heading': heading,
                                  'speed': speed,
                                  'vel': (speed * math.cos(heading), speed * math.sin(heading)),
                                  'bb': (bb_x, bb_y)
                                  }
                    ego_list[cur_step] = agent_dict
                elif " pedestrians" in line:  # exo_car info start
                    exo_count = int(line.split(' ')[0])
                    exos_list[cur_step] = []
                elif "id / pos / speed / vel / intention / dist2car / infront" in line:  # exo line, info start from index 16
                    # agent 0: id / pos / speed / vel / intention / dist2car / infront =  54288 / (99.732, 462.65) / 1 / (-1.8831, 3.3379) / -1 / 9.4447 / 0 (mode) 1 (type) 0 (bb) 0.90993 2.1039 (cross) 1 (heading) 2.0874
                    line_split = line.split(' ')
                    agent_id = int(line_split[16 + 1])

                    pos_x = float(line_split[18 + 1].replace('(', '').replace(',', ''))
                    pos_y = float(line_split[19 + 1].replace(')', '').replace(',', ''))
                    pos = [pos_x, pos_y]
                    vel_x = float(line_split[23 + 1].replace('(', '').replace(',', ''))
                    vel_y = float(line_split[24 + 1].replace(')', '').replace(',', ''))
                    vel = [vel_x, vel_y]
                    bb_x = float(line_split[36 + 1])
                    bb_y = float(line_split[37 + 1])
                    heading = float(line_split[41 + 1])
                    agent_dict = {'id': agent_id,
                                  'pos': [pos_x, pos_y],
                                  'heading': heading,
                                  'vel': [vel_x, vel_y],
                                  'bb': (bb_x * 2, bb_y * 2)
                                  }

                    exos_list[cur_step].append(agent_dict)
                    assert (len(exos_list[cur_step]) <= exo_count)
                elif "Path: " in line:  # path info
                    # Path: 95.166 470.81 95.141 470.86 ...
                    line_split = line.split(' ')
                    path = []
                    for i in range(1, len(line_split) - 1, 2):
                        x = float(line_split[i])
                        y = float(line_split[i + 1])
                        path.append([x, y])
                    ego_path_list[cur_step] = path
                elif 'predicted_car_' in line:
                    # predicted_car_0 378.632 470.888 5.541
                    # (x, y, heading in rad)
                    line_split = line.split(' ')
                    pred_step = int(line_split[0][14:])
                    x = float(line_split[1])
                    y = float(line_split[2])
                    heading = float(line_split[3])
                    agent_dict = {'pos': [x, y],
                                  'heading': heading,
                                  'bb': (10.0, 10.0)
                                  }
                    if pred_step == 0:
                        pred_car_list[cur_step] = []
                    pred_car_list[cur_step].append(agent_dict)

                elif 'predicted_agents_' in line:
                    # predicted_agents_0 380.443 474.335 5.5686 0.383117 1.1751
                    # [(x, y, heading, bb_x, bb_y)]
                    line_split = line.split(' ')
                    if line_split[-1] == "" or line_split[-1] == "\n":
                        line_split = line_split[:-1]
                    pred_step = int(line_split[0][17:])
                    if pred_step == 0:
                        pred_exo_list[cur_step] = []
                    num_agents = (len(line_split) - 1) / 5
                    agent_list = []
                    for i in range(int(num_agents)):
                        start = 1 + i * 5
                        x = float(line_split[start])
                        y = float(line_split[start + 1])
                        heading = float(line_split[start + 2])
                        bb_x = float(line_split[start + 3])
                        bb_y = float(line_split[start + 4])
                        agent_dict = {'pos': [x, y],
                                      'heading': heading,
                                      'bb': (bb_x * 2, bb_y * 2)
                                      }
                        agent_list.append(agent_dict)
                    pred_exo_list[cur_step].append(agent_list)
                elif 'INFO: Executing action' in line:
                    line_split = line.split(' ')
                    steer = float(line_split[5].split('/')[0])
                    acc = float(line_split[5].split('/')[1])
                    speed = float(line_split[5].split('/')[2])
                    action_list[cur_step] = (steer, acc, speed)
                    # INFO: Executing action:22 steer/acc = 0/3
                elif "Trials: no. / max length" in line:
                    line_split = line.split(' ')
                    trial = int(line_split[6])
                    depth = int(line_split[8])
                    trial_list[cur_step] = trial
                    depth_list[cur_step] = depth
                if 'collision = 1' in line or 'INININ' in line or 'in real collision' in line:
                    coll_bool_list[cur_step] = 1

            except Exception as e:
                error_handler(e)
                pdb.set_trace()

    print('done')

    return action_list, ego_list, ego_path_list, exos_list, coll_bool_list, pred_car_list, pred_exo_list, trial_list, depth_list

def progress(ego_list):
    time =[]
    pre,suc = 0,0
    for key in ego_list.keys():
        if np.abs(ego_list[key]['speed'])>2:
            pre = 1
        elif np.abs(ego_list[key]['speed'])<0.2:
            suc = 1
            if pre == 1:
                time.append(int(key)-1)
                pre=suc=0
        else:
            pre=suc=0
    return time

def progress2(ego_list):
    time =[]
    for key in ego_list.keys():
        if key+1 in ego_list.keys():
            pre = ego_list[key]['speed']
            suc = ego_list[key+1]['speed']
            if pre-suc > 3:
                time.append(key)
    return time

def list_all_files(rootdir):
    _files = []
    listdir = os.listdir(rootdir)
    for i in range(len(listdir)):
        path = os.path.join(rootdir, listdir[i])
        if os.path.isdir(path):
            _files.extend(list_all_files(path))
        if os.path.isfile(path):
            _files.append(path)
    return _files

In [ ]:
if __name__ == "__main__":
    
    root = os.path.dirname(__file__)
    
    FOLDER_PATHS = [
        'hivt1hz_t0_03_slowdownHz30times\\',
        'lanegcn1hz_t0_03_slowdownHz30times/',
        'knndefault_3Hz_ts0_1_allHzscale/',
        'knnsocial_3Hz_ts0_1_allHzscale/',
        'lstmdefault_3Hz_ts0_1_allHzscale/',
        'lstmsocial_3Hz_ts0_1_allHzscale/',
        'cv10hz_t0_33_slowdownHz3times/', 
        'ca10hz_t0_33_slowdownHz3times/',
    ]


    distributions = {}
    velo = {}
    ADE_d = {}
    ADE_d_Closest = {}
    Average_velocity = {}
    mean = {}
    variance = {}
    pred = 10

    ADE = []
    ADE_CLOSE = []

    for folder_path in FOLDER_PATHS:
        all_files = list_all_files(os.path.join(root,folder_path))
        experiment_name = folder_path.split('_')[0]
        distributions[experiment_name] = []
        velo[experiment_name] = []
        Average_velocity[experiment_name] = []

        for myfile in all_files:
            ADE_d[experiment_name] = []
            ADE_d_Closest[experiment_name] = []
            _, ego_list, _, exos_list, _, pred_car_list, pred_exo_list, _, _ = parse_data(myfile)

            if len(ego_list) >= 80:
            
                print("Method: ", myfile.split("\\")[1].split("_")[0])
                print("Total length:", len(ego_list))

                for t in range(len(ego_list)-pred):
                    exo_pos_list = exos_list[t]
                    pred_exo_pos_list = pred_exo_list[t]

                    # find pred and ground-truth of each agent
                    ade_1 = []
                    for i in range(len(exo_pos_list)):
                        agent_id = exo_pos_list[i]['id']
                        gt_pos_list = []
                        for next_t in range(t, t+pred):
                            all_agent_ids_of_next_t = [agent['id'] for agent in exos_list[next_t]]
                            if agent_id in all_agent_ids_of_next_t:
                                gt_pos_list.append(exos_list[next_t][all_agent_ids_of_next_t.index(agent_id)]['pos'])
                        # if enough 30, then find the distance
                        if len(gt_pos_list) == pred:
                            # need to find prediction of this agent id
                            pred_pos_list = []
                            all_agent_ids_of_t = [agent['id'] for agent in exos_list[t]]
                            for next_t in range(pred):
                                pred_pos_list.append(pred_exo_pos_list[next_t][all_agent_ids_of_t.index(agent_id)]['pos'])
                            dist = np.array(gt_pos_list) - np.array(pred_pos_list)
                            dist = np.sqrt((dist**2).sum(1))
                            distributions[experiment_name].append(np.mean(dist))
                            ade_1.append(np.mean(dist))
                    ade_d = np.array(np.mean(ade_1)) if len(ade_1) !=0 else 0
                    ade_closest = ade_1[0] if len(ade_1) !=0 else 0
                    ADE_d[experiment_name].append(ade_d)
                    ADE_d_Closest[experiment_name].append(ade_closest)                
            else:
                continue
            ADE.append(np.mean(np.array(ADE_d[experiment_name])))
            ADE_CLOSE.append(np.mean(np.array(ADE_d_Closest[experiment_name])))

    fig = plt.figure()
    plt.plot(2,2,1)
    plt.scatter(ADE, stop_freq, color= 'red')
    plt.xlabel("ADE(K=1)")
    plt.ylabel("Time Interval of Sharp Deceleration")
    plt.xlim(0,1.2)
    plt.ylim(0,350)

    plt.subplot(2,2,2)
    plt.scatter(ADE_CLOSE, stop_freq, color= 'red')
    plt.xlabel("ADE for the Closest")
    plt.ylabel("Time Interval of Sharp Deceleration")
    plt.subplot(2,2,3)
    plt.scatter(ADE, stop_dis, color= 'red')
    plt.xlabel("ADE")
    plt.ylabel("Distance of Sharp Deceleration")
    plt.subplot(2,2,4)
    plt.scatter(ADE_CLOSE, stop_dis, color= 'red')
    plt.xlabel("ADE for the Closest")
    plt.ylabel("Distance of Sharp Deceleration")

    plt.show()

    # plot the distribution by histogram in a 4x4 subplot
    # fig, axs = plt.subplots(2, 4, figsize=(15,30))
    # colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22',
    #           '#17becf']
    # sequence = [7,0,4,5,3,6,1,2]

    # for i, key in enumerate(distributions.keys()):
    #     if key != "knnsocial":
    #         gm = GaussianMixture(n_components=2, random_state=0).fit(np.array(distributions[key]).reshape(-1, 1))
    #     else:
    #         gm = GaussianMixture(n_components=3, random_state=0).fit(np.array(distributions[key]).reshape(-1, 1))
    #     mean[key] = gm.means_
    #     variance[key] = gm.covariances_.reshape(-1,1)
    #     print("Number of Data:", np.array(distributions[key]).shape)

    #     ax = axs[sequence[i]//4, sequence[i]%4]
    #     ax.hist(np.array(distributions[key]), bins=100, color=colors[i])
    #     # ax.set_xlim(0, 15)
    #     ax.set_title(key)
    #     if sequence[i]>=4:
    #         ax.set_xlabel('ADE')
    #     ax.set_ylabel('frequency')
    # plt.show()

    # fig = plt.figure()
    # method = list(velo.keys())
    # plot_speed = []
    # plot_variance = []
    # for i in range(8):
    #     x_key = method[sequence.index(i)]
    #     plot_speed.append(np.mean(np.array(velo[x_key])))
    #     plot_variance.append(np.mean(variance[x_key]))
    #     # y_velo = np.mean(np.array(velo[x_key]))
    #     # plt.bar(x_key,y_velo, color = colors[sequence.index(i)])
    #     # print("Average Velocity for method %s is %s" % (x_key, y_velo))
    #     # std_mean = variance[x_key]
    #     # print("Std for method %s is %s" % (x_key, std_mean))
    # plt.scatter(plot_variance,plot_speed, color = "red")
    # plt.xlabel("Variance")
    # plt.ylabel("Average Velocity")
    # plt.show()